

# 🛠️ **Interactive OpenAI Function Calling Tutorial** 

earn by doing! This interactive tutorial lets walks through creating a function and then you can chat with the Assistant to use it.

Get an intuitive understanding of Function Calling in under 5 mins!!

---

## 🤔 **What is Function Calling?**

Function calling is a powerful feature that empowers ChatGPT to *act*. By defining specific abilities (or functions), you grant ChatGPT the capability to perform them based on user prompts.

---

## 🚀 **What to Expect in this Notebook**

In this notebook, we'll look at examples to understand how function calling operates:

1. 🎈 **Simple Example:** Balloon launch function
2. 🌦️ **Advanced Example:** Fetch weather info function

---

### 🛠️ **Get Creative!**
This notebook is easy to modify. Feel free to use this as a base to build your own functions and experiment 🧪

### 🛠️ **About**
👤 **Creator:** [Josh Bickett](https://twitter.com/josh_bickett)

📘 **Colab:** [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joshbickett/function-calling-notebook/blob/main/interactive-function-calling-notebook.ipynb)

🔗 **Github:** [View on GitHub](https://github.com/joshbickett/function-calling-notebook)



In [ ]:
!pip install openai

In [3]:

import requests
import openai
import json
import random

In [4]:
openai.api_key = 'YOUR_API_KEY'

In [15]:

def launch_water_balloon(balloon_size):
    hit_locations = ['the ground', 'a tree', 'the moon']
    random_location = random.choice(hit_locations)
    launch_results = f'Launching the {balloon_size} water balloon, it hit {random_location}!'
    return launch_results


In [6]:
functions = [
    {
        "name": "launch_water_balloon",
        "description": "This function launches a hypothetical water balloon",
        "parameters": {
            "type": "object",
            "properties": {
                "balloon_size": {
                    "type": "string",
                    "enum": ["Large", "Medium", "Small"],
                },
            },
            "required": ["balloon_size"],
        },
    }
]

available_functions = {
    "launch_water_balloon": launch_water_balloon,
}

In [ ]:
INSPECT = True

In [13]:

def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        if INSPECT:
          print('[inspector] Oh, we got a function call! 🔨')
          print('[inspector] ===> function_name', function_name)
          print('[inspector] ===> function_args', function_args)
        function_response = function_to_call(
            balloon_size=function_args.get("balloon_size"),
        )
        if INSPECT:
          print('[inspector] ===> function_response', function_response)


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

    

      

In [ ]:
messages = []
message_count = 0 
while message_count < 10: 
    user_input = input("User: ")
    user_message = {"role": "user", "content": user_input}
    messages.append(user_message)
    converse(messages)
    message_count += 1

In [ ]:



cities = [
    {"name": "San Francisco", "latitude": 37.7749, "longitude": -122.4194},
    {"name": "New York City", "latitude": 40.7128, "longitude": -74.0060},
    {"name": "Los Angeles", "latitude": 34.0522, "longitude": -118.2437},
    {"name": "Chicago", "latitude": 41.8781, "longitude": -87.6298},
    {"name": "Miami", "latitude": 25.7617, "longitude": -80.1918}
]

def get_weather_data(city_name):
    # Find the city object by name
    city = next((c for c in cities if c["name"] == city_name), None)

    # If city is found, get the weather data using its coordinates
    if city:
        # Construct the URL based on the city's latitude and longitude
        url = f"https://api.weather.gov/points/{city['latitude']},{city['longitude']}"

        # Make a request to the URL
        response = requests.get(url)
        

        # Ensure the response status is 200 (OK)
        if response.status_code == 200:
            data = response.json()

            # Extracting the forecast URL from the response
            forecast_url = data["properties"]["forecast"]

            # Fetch the actual forecast data using the extracted URL
            forecast_response = requests.get(forecast_url)

            if forecast_response.status_code == 200:
                forecast_data = forecast_response.json()
                tonight_forecast = forecast_data['properties']['periods'][0]

                # Display the forecast
                return tonight_forecast['detailedForecast']
            else:
                print(f"Error fetching forecast data. HTTP Status Code: {forecast_response.status_code}")
                return None
        else:
            print(f"Error fetching grid point data. HTTP Status Code: {response.status_code}")
            return None
    else:
        print(f"City '{city_name}' not found in the list.")
        return None


In [ ]:
functions = [{
    "name": "get_weather_data",
    "description": "This function can get weather information for some of the US's largest cities",
    "parameters": {
        "type": "object",
        "properties": {
            "city_name": {
                "type": "string",
                "enum": ["San Francisco", "New York City", "Los Angeles", "Chicago", "Miami"],
            },
        },
        "required": ["city_name"],
    },
}]
available_functions = {
    "get_weather_data": get_weather_data,
}

In [ ]:

def converse(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]
    
    
    messages.append(response_message) 
    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        if INSPECT:
          print('[inspector] Oh, we got a function call! 🔨')
          print('[inspector] ===> function_name', function_name)
          print('[inspector] ===> function_args', function_args)
        function_response = function_to_call(
            city_name=function_args.get("city_name"),
        )
        if INSPECT:
          print('[inspector] ===> function_response', function_response)


        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        print('Assistant: ', function_response)
        
    else: 
        content = response_message.get("content")
        print('Assistant: ', content)

    

      

In [ ]:
messages = []
message_count = 0 
while message_count < 10: 
    user_input = input("User: ")
    user_message = {"role": "user", "content": user_input}
    messages.append(user_message)
    converse(messages)
    message_count += 1